In [1]:
#DataFlair Project
#import all the required libraries
import numpy as np
import pandas as pd
import pickle
from statistics import mode
import nltk
from nltk import word_tokenize
from nltk.stem import LancasterStemmer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,LSTM,Embedding,Dense,Concatenate,Attention
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup

[nltk_data] Downloading package wordnet to C:\Users\Subhan
[nltk_data]     Sheikh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Subhan
[nltk_data]     Sheikh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Subhan
[nltk_data]     Sheikh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df=pd.read_csv("./data/Reviews.csv",nrows=100000)
df.head()

In [ ]:
df.dropna(axis = 0,inplace=True)

# subset=['Text']:
# Meaning: Specifies that duplicates should be identified based only on the 'Text' column.
df.drop_duplicates(subset=['Text'],inplace=True)
input_data = df.loc[:,'Text']
target_data = df.loc[:,'Summary']

# Purpose: This line of code scans through the DataFrame (or Series) target and replaces every occurrence of an empty string
# ('') with a NaN value (np.nan).
target_data.replace('',np.nan,inplace=True)

In [ ]:
df.columns

In [ ]:
input_data

In [ ]:
# !pip install contractions
# import contractions
# Example string with contractions
# text = "I'm learning how to use contractions. It's a useful tool, isn't it?"
# # Expand contractions
# expanded_text = contractions.fix(text)
# print(expanded_text)

# Access the internal contraction dictionary


In [ ]:
input_texts = []
target_texts = []
input_words = []
target_words = []
stop_words = set(stopwords.words('english'))
stemm=LancasterStemmer()

In [ ]:
stop_words

In [ ]:
# The code snippet texts = BeautifulSoup(texts, "lxml").text uses the BeautifulSoup 
# library to parse HTML or XML content and extract the text from it. Let's break 
# down the purpose and functionality of each part of the code:
import contractions
def clean(texts,src):
    texts = BeautifulSoup(texts, "lxml").text
    texts = texts.replace("no", "")
    words = word_tokenize(texts.lower())
    words = list(filter(lambda w: (w.isalpha() and len(w)>=3),words))
    words = [contractions.fix(w) for w in words]
    if src == "input":
        words = [stemm.stem(w) for w in words if w not in stop_words]
    else:
        words = [w for w in words if w not in stop_words]

    return words


In [ ]:
clean("I'm learning how to use haven't contractions 123. It's a useful tool, isn't it?","input")

In [ ]:
# Usage: Typically, "sos" is added to the beginning of the target sequence to
#     prompt the model to start generating the target text.
# Usage: "eos" is added to the end of the target sequence to indicate where the 
#     model should stop generating text.
for in_text,tr_text in zip(input_data,target_data):
    
    in_words= clean(in_text,"input")
    input_texts += [" ".join(in_words)]
    input_words += in_words
    tr_words = clean("sos" + tr_text + "eos","target")
    target_texts +=[" ".join(tr_words)]
    target_words += tr_words
print("Gfg")

In [ ]:
# target_texts

In [ ]:
# input_words

In [ ]:
input_words = sorted(list(set(input_words)))
target_words = sorted(list(set(target_words)))
num_in_words = len(input_words)
num_tr_words = len(target_words)

# .)Now se max len of input and target text.
max_in_len = mode([len(i) for i in input_texts])
max_tr_len = mode([len(i) for i in target_texts])

print("number of input words : ",num_in_words)
print("number of target words : ",num_tr_words)
print("maximum input length : ",max_in_len)
print("maximum target length : ",max_tr_len)

In [ ]:
#split the input and target text into 80:20 ratio or testing size of 20%.
x_train,x_test,y_train,y_test=train_test_split(input_texts,target_texts,test_size=0.2,random_state=0)

In [ ]:
#train the tokenizer with all the words
in_tokenizer = Tokenizer()
in_tokenizer.fit_on_texts(x_train)
tr_tokenizer = Tokenizer()
tr_tokenizer.fit_on_texts(y_train)
 
#convert text into sequence of integers
#where the integer will be the index of that word
x_train= in_tokenizer.texts_to_sequences(x_train) 
y_train= tr_tokenizer.texts_to_sequences(y_train)

In [ ]:
# x_train

In [ ]:
# y_train

In [ ]:
en_in_data = pad_sequences(x_train,maxlen = max_in_len,padding="post")
dec_data = pad_sequences(y_train,maxlen = max_tr_len,padding="post")

In [ ]:
# Decoder Input Data (dec_in_data):

# The decoder input data (dec_in_data) represents the input sequences for the decoder,
# excluding the last word (which is "eos" in this case). This is because during training,
# the decoder is fed with its own output from the previous time step as input. So,
# the last word ("eos") is removed to ensure that the model does not receive the
# "eos" token as input during training.

# Decoder Target Data (dec_tr_data):

# The decoder target data (dec_tr_data) represents the target sequences for the decoder, 
# excluding the first word (which is "sos" in this case). This is because during 
# training, the model is trained to predict the next word in the sequence given
# the previous words as input. So, the first word ("sos") is removed to ensure
# that the model predicts the correct words in the sequence.



# dec_data: This is the array representing the padded sequences of the decoder input data.
#     Each row corresponds to a sequence, and each column represents a time step in 
#     the sequence.
# [:, :-1]: This slicing operation selects all rows (:) and all columns up to the
#     second-to-last column (:-1). It excludes the last column from each row.
dec_in_data = dec_data[:,:-1]
    
#     dec_data.reshape(len(dec_data), max_tr_len, 1): This reshapes dec_data into a 3D 
#         array with dimensions (len(dec_data), max_tr_len, 1). The second dimension 
#         (max_tr_len) represents the maximum target length, and the third dimension
#         (1) is added to make it compatible with slicing.
# [:, 1:]: This slicing operation selects all rows and all columns starting from the second 
#     column (index 1) until the end.
dec_tr_data = dec_data.reshape(len(dec_data),max_tr_len,1)[:,1:]
    

In [ ]:
K.clear_session() 
latent_dim = 500
 
#create input object of total number of encoder words
en_inputs = Input(shape=(max_in_len,)) 

# Purpose: This layer converts word indices into dense vectors of size latent_dim.
#     It looks up an embedding matrix to convert each word index into a
#     latent_dim-dimensional vector.
# Input: The word indices from en_inputs.
# Output: Dense vectors of size (max_in_len, latent_dim) for each input sequence.
en_embedding = Embedding(num_in_words+1, latent_dim)(en_inputs) 

In [ ]:
# Purpose: These three stacked LSTM layers process the input sequences to capture
#     temporal dependencies. Each LSTM layer takes the sequence output from the 
#     previous layer and produces a new sequence of the same length (max_in_len),
#     but with updated hidden states.
# Input: The output sequence from the previous layer (or embedding layer for the first LSTM).
# Output: For each LSTM layer:
# en_outputsX: The output sequences for each time step of the input 
#     (shape: (batch_size, max_in_len, latent_dim)).
# state_hX: The hidden state at the last time step (shape: (batch_size, latent_dim)).
# state_cX: The cell state at the last time step (shape: (batch_size, latent_dim)
                                                
                                                
en_lstm1= LSTM(latent_dim, return_state=True, return_sequences=True) 
en_outputs1, state_h1, state_c1= en_lstm1(en_embedding) 
 
#LSTM2
en_lstm2= LSTM(latent_dim, return_state=True, return_sequences=True)   
en_outputs2, state_h2, state_c2= en_lstm2(en_outputs1) 
 
#LSTM3
en_lstm3= LSTM(latent_dim,return_sequences=True,return_state=True)
en_outputs3 , state_h3 , state_c3= en_lstm3(en_outputs2)
 
# Purpose: The final hidden and cell states of the last LSTM layer in the encoder
# are passed to the decoder as initial states to provide context for generating the
# target sequence.
# Output: A list containing state_h3 and state_c3.
en_states= [state_h3, state_c3]

In [ ]:
# Decoder. 

# Purpose: This layer specifies the shape of the decoder input, which is a
#     sequence of word indices with an undefined length (None), allowing for 
#     flexible sequence lengths during training and inference.
# Input: A batch of sequences of word indices (target sequences).
dec_inputs = Input(shape=(None,)) 

# Purpose: Similar to the encoder embedding layer, it converts target word indices 
#     into dense vectors of size latent_dim.
# Input: The word indices from dec_inputs.
# Output: Dense vectors for the target sequences (shape: (batch_size, target_sequence_length,
#                                                         latent_dim)).
    
dec_emb_layer = Embedding(num_tr_words+1, latent_dim) 
dec_embedding = dec_emb_layer(dec_inputs) 
 
    
# Purpose: This LSTM layer processes the embedded target sequences, using the final states
#     of the encoder LSTM as its initial states to maintain context.
# Input: The embedded target sequences and the initial states from the encoder (en_states).
# Output:
# dec_outputs: The output sequences for each time step of the input (shape: (batch_size,
#                                     target_sequence_length, latent_dim)).
# The hidden and cell states at the last time step (not used further in this case, hence _).

dec_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
dec_outputs, *_ = dec_lstm(dec_embedding,initial_state=en_states) 

In [ ]:
#Attention layer

# Purpose: The attention layer allows the decoder to focus on different parts of the 
#     encoder's output sequence when generating each word in the target sequence
#     It computes the context vector as a weighted sum of the encoder output 
#     sequences, where the weights are determined by the alignment scores between 
#     the decoder and encoder outputs.
# Input:
# dec_outputs: The output sequences from the decoder LSTM (query).
# en_outputs3: The output sequences from the last encoder LSTM (value and key).
# Output:
# attn_out: The context vector computed by the attention mechanism 
#     (shape: (batch_size, target_sequence_length, latent_dim)).

attention =Attention()
attn_out = attention([dec_outputs,en_outputs3])

 
# Purpose: This layer concatenates the attention output (attn_out) with the decoder 
#     outputs (dec_outputs) along the last axis to combine context information from 
#     the encoder with the current state of the decoder.
# Input:
# dec_outputs: The output sequences from the decoder LSTM.
# attn_out: The context vector from the attention layer.
# Output: A combined representation of the decoder output and attention context 
#     (shape: (batch_size, target_sequence_length, 2 * latent_dim)).
merge=Concatenate(axis=-1, name='concat_layer1')([dec_outputs,attn_out])

In [ ]:
# Purpose: This layer produces the final output probabilities for each word
#     in the target vocabulary. It uses the softmax activation function
#     to generate a probability distribution over all possible words in the vocabulary.
# Input: The concatenated output from the merge layer.
# Output: Probability distributions over the target vocabulary for each time step 
#     of the target sequence (shape: (batch_size, target_sequence_length, num_tr_words + 1))
    
#Dense layer (output layer)
dec_dense = Dense(num_tr_words+1, activation='softmax') 
dec_outputs = dec_dense(merge) 

In [ ]:
# #Model class and model summary for text Summarizer

# Purpose: This defines the full sequence-to-sequence model with attention.
#     It takes the encoder and decoder inputs and produces the final target sequence 
#     probabilities.
# Input:
# en_inputs: The input sequences for the encoder.
# dec_inputs: The input sequences for the decoder.
# Output: The probability distributions over the target vocabulary for each time step
#     of the target sequence.
    
model = Model([en_inputs, dec_inputs], dec_outputs) 
model.summary()

In [ ]:
model.compile( 
    optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"] ) 
model.fit( 
    [en_in_data, dec_in_data],
    dec_tr_data, 
    batch_size=512, 
    epochs=10, 
    validation_split=0.1,
    )
 
#Save model
model.save("s2s")